In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Movies_metadata verilerinden ilgileneceğimiz başlıkları seçtik.

In [ ]:
df = pd.read_csv("/content/movies_metadata.csv",  usecols=['title', 'overview', ], low_memory=False)

In [ ]:
df.head()

,overview,title
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,When siblings Judy and Peter discover an encha...,Jumanji
2,A family wedding reignites the ancient feud be...,Grumpier Old Men
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale
4,Just when George Banks has recovered from his ...,Father of the Bride Part II


In [ ]:
df.shape

(45466, 2)

Ram'da fazla yer kaplamamak adına örneklemin %10'nunu alalım.

In [ ]:
df = df.sample(frac=0.1)

In [ ]:
df.shape

(4547, 2)

#**Metin vektörleştirme (TFI-IDF)**

In [ ]:
df["overview"].head()

34054    Two young couples are driving through a remote...
35931    Willy Ferriere is dead broke and his mistress ...
39251    Mickey is trying to lead a concert of The Will...
7257     The host of an investigative news show is conv...
21841    A reimagining of the classic horror tale about...
Name: overview, dtype: object

In [ ]:
tfidf = TfidfVectorizer(stop_words="english")

In [ ]:
df['overview'] = df['overview'].fillna('')

In [ ]:
tfidf_matrix = tfidf.fit_transform(df['overview'])


In [ ]:
tfidf_matrix.shape

(4547, 23900)

In [ ]:
df['title'].shape

(4547,)

In [ ]:
tfidf.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '000th',
 '006',
 '007',
 '008',
 '009',
 '01',
 '02',
 '03',
 '04',
 '042',
 '05',
 '06',
 '07',
 '08',
 '088',
 '09',
 '10',
 '100',
 '1000',
 '1001',
 '100th',
 '101',
 '105',
 '108',
 '10th',
 '11',
 '115',
 '117',
 '11th',
 '12',
 '1200',
 '125',
 '12th',
 '13',
 '130',
 '1349',
 '13anos',
 '13b',
 '13th',
 '14',
 '142',
 '1492',
 '14th',
 '15',
 '150',
 '150th',
 '1518',
 '15th',
 '16',
 '1600',
 '161',
 '1616',
 '1684',
 '16th',
 '17',
 '1700',
 '1701',
 '174',
 '1745',
 '1746',
 '1793',
 '1794',
 '1797',
 '17th',
 '18',
 '1800',
 '1800s',
 '1820',
 '1820s',
 '1826',
 '1828',
 '1840s',
 '1841',
 '1846',
 '1848',
 '1849',
 '1850',
 '1852',
 '1857',
 '1860',
 '1860s',
 '1861',
 '1862',
 '1864',
 '1865',
 '1867',
 '1870',
 '1870s',
 '1880s',
 '1882',
 '1885',
 '1890',
 '1890s',
 '1899',
 '18th',
 '19',
 '1900',
 '1900s',
 '1902',
 '1903',
 '1904',
 '1905',
 '1906',
 '1910',
 '1911',
 '1912',
 '1913',
 '1914',
 '1915',
 '1916',
 '1917',
 '1918',
 '192',
 '1920',
 '19

In [ ]:
tfidf_matrix.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.14605738, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

#**Cosine Sim Calculator**

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)

In [ ]:
cosine_sim.shape

(4547, 4547)

In [ ]:
cosine_sim[1]

array([0.        , 1.        , 0.        , ..., 0.01933347, 0.        ,
       0.        ])

#** Recommendation Based on Similarities **

In [ ]:
indices = pd.Series(df.index, index=df['title'])

In [ ]:
indices.index.value_counts()

Alice in Wonderland           3
Sense and Sensibility         3
A Christmas Carol             3
Beautiful                     2
Dementia                      2
                             ..
The Burglars                  1
Follow                        1
Love! Valour! Compassion!     1
Ziegfeld Follies              1
The Murder of Fred Hampton    1
Name: title, Length: 4503, dtype: int64

In [ ]:
indices = indices[~indices.index.duplicated(keep='last')]

In [ ]:
indices["Cinderella"]

35595

In [ ]:
movie_index = indices["Cinderella"]

In [ ]:
valid_indices = range(0, 4547)
for movie_index in valid_indices:
    value = cosine_sim[movie_index]

In [ ]:
similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"])


In [ ]:
movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [ ]:
df['title'].iloc[movie_indices]

3065     Mr. Death: The Rise and Fall of Fred A. Leucht...
21695                                                Saint
9535                                   Hellraiser: Inferno
19955                                      The Awful Truth
24922                                 Thesis on a Homicide
24008                                         King's Faith
3928                                   Save the Last Dance
16367                                               Stalin
11513                                     American Madness
44199                      The Velvet Underground and Nico
Name: title, dtype: object

In [ ]:
def content_based_recommender(title, cosine_sim, dataframe):
    # index'leri olusturma
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # title'ın index'ini yakalama
    movie_index = indices[title]
    # title'a gore benzerlik skorlarını hesapalama
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    # kendisi haric ilk 10 filmi getirme
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]

In [ ]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim



In [ ]:
cosine_sim = calculate_cosine_sim(df)


In [ ]:
content_based_recommender('Save the Last Dance', cosine_sim, df)

32414                         Attack on Titan
217                              Castle Freak
28806                              Shark Kill
17752                       Frenchman's Creek
31017    Axe Giant - The Wrath of Paul Bunyan
7371                      Witchfinder General
10025                                   Earth
29674                      Barbie: Fairytopia
42451                    King of the Belgians
27037        Snow White and the Three Stooges
Name: title, dtype: object